<a href="https://colab.research.google.com/github/meenagowrishankar/code_demo_qed/blob/main/H2_encoded_Quantinuum_post_processing_savedresults_V2_0_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Program for post-processing results from running a 6 qubit, [[4,2,2]] hydrogen vqe with post-selection of the counts based on ancilla 1 = 0, logical state parity, and based on both ancilla 1 = 0 and logical state parity, and separating expectation values by value of ancilla 2 measurement. <br>
**Bit order LSB for Quantinuum Emulator: $|q_nq_{n-1}...q_0\rangle$, regardless of order of measurement in circuit, results are reordered in the output file in the order mentioned above.**

In [ ]:
# import xacc


In [1]:
import json
import math
import numpy as np
import pickle
from collections import Counter

**Functions for post-processing data**

In [2]:
# Separate the parallel circuit counts into individual circuit counts

def sep_counts(counts:dict) -> dict:
    '''
    Returns tuples of tuples of bitstring and measurements of each circuit
    within a parallel execution of three circuits.
    Takes a dictionary, counts, with bitstrings as keys and
    returns list of tuples with six bit bitstrings as keys with corresponding
    values of the origial key in the original counts dictionary. Tuples used because
    the resulting six bit bitstrings are non-unique.

        Parameters:
            counts (dict) : counts for all measured bitstrings
            MSB (bool): determines the bit order (defaults to True (MSB))
    '''

    all_counts = dict(counts)

    for key in all_counts.keys():
#         print("key: ", all_counts)
        list1 = tuple((k[:6],v) for k,v in all_counts.items())
        list2 = tuple((k[12:],v) for k,v in all_counts.items())
        list3 = tuple((k[6:12],v) for k,v in all_counts.items())

#     print(list1,list2,list3)

    return list1, list2, list3


def combine_counts(counts:dict)->dict:
    '''
    Returns a measurement counts dictionary by combining the counts of three circuits run in parallel.
    Parameters: dictionary with the counts of bitstring for three circuits run in parallel.
    Counts is a tuple of tuples. Each tuple within tuple consists of a pair, with first element of the
    tuple being the two bit bitstring and the second element, the associated measurement value of the bitstring.
    Returns dictionary with keys as the two bit bitstring, and as value, the total measurement counts of the corresponding
    two bit bistring obtained from the sum of the second element of each tuple that has as its
    first element the bitstring key of the dictionary.
    '''

    # separate the whole bitstring into bitstrngs of each circuit and assign the correspinding value
    # of the whole bitstring to each of the split bitstrings
    counts1, counts2, counts3 = sep_counts(counts)

    # dictionary to split counts for non-unique bitstrings
    counts_dict1 = {}
    counts_dict2 = {}
    counts_dict3 = {}

    # Create dictionary for each key in each of the two dictionaries
    for key, value in counts2:
#         print(key)
        counts_dict2[key] = 0

    for key, value in counts1:
        counts_dict1[key] = 0

    for key, value in counts3:
        counts_dict3[key] = 0

    # Add counts for each non-unique bitstring in each of the two dictionaries
    for key, value in counts2:
        if key in counts_dict2.keys():
            counts_dict2[key] += value
    for key, value in counts1:
        if key in counts_dict1.keys():
            counts_dict1[key]+=value

    for key, value in counts3:
        if key in counts_dict3.keys():
            counts_dict3[key]+=value

#     counts_dict1, counts_dict2

    counts_dict = {}
    for key, value in counts_dict1.items():
        if key in counts_dict.keys():
            counts_dict[key] += counts_dict1[key]

        else:
            counts_dict[key] = counts_dict1[key]

    for key, value in counts_dict2.items():
        if key in counts_dict.keys():
            counts_dict[key] += counts_dict2[key]

        else:
            counts_dict[key] = counts_dict2[key]

    for key, value in counts_dict3.items():
        if key in counts_dict.keys():
            counts_dict[key] += counts_dict3[key]

        else:
            counts_dict[key] = counts_dict3[key]

    return counts_dict

def counts_list(counts:list, point, pauli):
    '''
    Takes list of dictionaries with six bit bitsrings and measurement counts for a Pauli term in the Hamiltonian,
    each dictionary is created from the corresponding file with results of a single parallel execution of
    six qubit circuits for a certain number of shots. Returns list of dictionaries for the specific Pauli
    term from each to be combined using meta_combine function.
    '''
    counts_list = []
    original_list = []
    for count in counts:
        counts_list.append(count[point][pauli]['MeasurementCounts'])
    return counts_list

def meta_combine(counts:list):
    '''
    Takes a list of dictionaries associated with a single Pauli term from the Hamiltonian
    within counts with multiple dictionaries of measurements and counts created from each file
    containing the results of each execution of parallel six-qubit circuits.
    Returns a single combined dictionary of bitstrings and measurement counts for use in calculating
    expectation values.
    '''
    comb_counts_dict = {}
    combine_counts = Counter()
    for count in counts:
        combine_counts+=Counter(count)
        comb_counts_dict = dict(combine_counts)

    return dict(comb_counts_dict)

In [3]:
# check if empty before combining counts
def check_and_combine(counts:dict)->dict:
    '''
    Check if dictionary is empty and then proceed with combining counts if not empty.
    '''
    if not counts:
        return {}
    else:
        return combine_counts(counts)
#     new_counts = combine_counts(counts)
#     return new_counts

In [4]:
# test for combine_counts, check_and_combine

newstuff = {"001100223322445566":4, "223322223322445566":3, '001100001100001100':2, '443344001100001100': 1}
# newstuff = {"001100":4, "223322":3, '001100':2, '443344': 1}
# for key in newstuff.keys():
#     print(key[12:])
combine_counts(newstuff)
# check_and_combine(newstuff)


{'001100': 12, '223322': 10, '443344': 1, '445566': 7}

In [5]:
# post selects based on the value of ancilla 1. Discards the counts with ancilla 1 = "1".

### q_6 is ancilla_1

def post_select_on_ancila_1(counts: dict, MSB: bool = True) -> dict:

    '''
    Returns a dictionary with counts post-selected on ancilla 0.

        Parameters:
            counts (dict): Counts for all measured bitstrings.
            MSB (bool): Determines the bit order (Defaults to True (MSB)).

        Returns:
            post_selected_counts (dict): Post-selected counts.
    '''

    post_selected_counts = dict(counts)
    for key in counts.keys():
        # if MSB order, then q0 is the first
        if key[0] == "1" and MSB:
            del post_selected_counts[key]
        # if LSB, then q0 is the last
        if key[-1] == "1" and not MSB:
            del post_selected_counts[key]

    return post_selected_counts

def post_select_codespace(counts: dict, MSB: bool = True)-> dict:

    '''
    Returns a dictionary with counts post-selected for bitstrings outside codespace for
    Pauli terms except for X2X3.

        Parameters:
            counts (dict): Ancilla 1 corrected counts for all measured bitstrings.
            MSB (bool): Determines the bit order (Defaults to True (MSB)).

        Returns:
            post_selected_codespace (dict): Post-selected bitstrings.
    '''
    # codespace defined by the bitstrings in the four physical basis states of the encoding
    codespace = ['0000', '1111', '1001', '0110', '1010', '0101', '0011', '1100']
    post_selected_codespace = dict(counts)
    for key in counts.keys():
        # if MSB order, then q0 is the first
        if key[2:] not in codespace and MSB:
            del post_selected_codespace[key]
        # if LSB, then q0 is the last
        if key[1:5] not in codespace and not MSB:
            del post_selected_codespace[key]


    return post_selected_codespace

# returns the parity of the bit string that is passed in to calculate
# the correct expectation value without considering the measure statements
# as additional Pauli terms as done right now by xacc/qpus.

def has_even_parity(bitstring: int) -> bool:

    '''
    Checks if bitstring is an even decimal number.

        Parameters:
            bitstring (int): bitstring in binary representation.

        Returns:
            parity (bool): Returns True if even and False if odd.
    '''

    count = 0
    b = 1
    parity = True
    for i in range(32):
        if (bitstring & (b << i)): count += 1
    if (count % 2): parity = False
    return parity




In [6]:
# Calculate expectation value for each pauli term in the Hamiltonian

def PauliEnergies(theta:float, counts:dict, MSB: bool = True):

    '''
    Takes as input each individual theta value and dictionary
    with "correcCounts" from the post_select_on_ancila_1 function
    for that theta
    Returns the dictionary counts with Pauli expectation values
    for ancilla 2 = 0 and ancilla 2 = 1 and separate counts for a2 = 0 and 1.
    '''
    PauliExpVal = {}
#     PauliExpVal[theta] = {}
    for key, pauli in counts.items():
#         print(key)
#         print(PauliExpVal)
        # for calculating expectation values as a sum for ancilla 2 = 0 and ancilla 2 = 1
        exp_value_theta = 0.0
        exp_val = 0.0

        # Separate the post-selected measurement counts into two dictionaries depending on the value of ancilla 2
        a_0 = {}
        a_1 = {}
        a0 = {}
        a1 = {}

        for bitstring in pauli['correctCounts'].keys():
            if bitstring[5] == '0' and MSB:

                a_0[bitstring] = pauli['correctCounts'][bitstring]

            if bitstring[5] == '1' and MSB:

                a_1[bitstring] = pauli['correctCounts'][bitstring]

            if bitstring[0] == '0' and not MSB:

                a_0[bitstring] = pauli['correctCounts'][bitstring]

            if bitstring[0] == '1' and not MSB:

                a_1[bitstring] = pauli['correctCounts'][bitstring]


        # for calculating expectation values as a sum for ancilla 2 = 0 and ancilla 2 = 1
        exp_value_theta = 0.0
        exp_val = 0.0

        # Find the parity of the bitstring in each dictionary by excluding the bits not part of the Pauli
        # strings in the Hamiltonian (q0 and q5) and assign the correct sign to the expectation values based
        # on the parity for LSB bit order

        for bitstr in a_0.keys():
            prob = a_0[bitstr]/sum(a_0.values())
            if key == 'z2z3' or key == 'x2x3':
                is_even = has_even_parity(int(bitstr[2:4], 2))
                if(not is_even):
                    prob = -prob
#                 print("prob_a1: ",key, prob)
                exp_val += prob
            if key == 'z1z2':
                is_even = has_even_parity(int(bitstr[3:5], 2))
#                 is_even = has_even_parity(int(bitstr[0:2], 2))
                if(not is_even):
                    prob = -prob
        #                 print("prob_a1: ",prob)
                exp_val += prob
            if key == 'z1z3':
                is_even = has_even_parity(int(bitstr[4]+bitstr[2], 2))
#                 is_even = has_even_parity(int(bitstr[0]+bitstr[2], 2))
                if(not is_even):
                    prob = -prob
        #                 print("prob_a1: ",prob)
                exp_val += prob


        for bitstr in a_1.keys():
            prob = a_1[bitstr]/sum(a_1.values())
            if key == 'z2z3' or key == 'x2x3':
                is_even = has_even_parity(int(bitstr[2:4], 2))
                if(not is_even):
                    prob = -prob
        #                 print("prob_a1: ",prob)
                exp_value_theta += prob
            if key == 'z1z2':
                is_even = has_even_parity(int(bitstr[3:5], 2))
#                 is_even = has_even_parity(int(bitstr[0:2], 2))
                if(not is_even):
                    prob = -prob
        #                 print("prob_a1: ",prob)
                exp_value_theta += prob
            if key == 'z1z3':
                is_even = has_even_parity(int(bitstr[4]+bitstr[2], 2))
#                 is_even = has_even_parity(int(bitstr[0]+bitstr[2], 2))
                if(not is_even):
                    prob = -prob
        #                 print("prob_a1: ",prob)
                exp_value_theta += prob



        coefficient = counts[key]['coefficient']
#         print('coeff ',coefficient)

        # Total for calculating probability of success
        total_counts = sum(counts[key]['MeasurementCounts'].values())
        total_a2_0 = sum(a_0.values())
        total_a2_1 = sum(a_1.values())
        counts[key]['a2_0 exp-val'] = exp_val
        counts[key]['a2_1 exp-val'] = exp_value_theta
        counts[key]['a2_0 counts'] = a_0
        counts[key]['a2_1 counts'] = a_1
        # Probability of success p0 for each a_2 measurement
        if key == 'I':
            counts[key]['p0_a20'] = 0.0
            counts[key]['p0_a21'] = 0.0
        else:
            counts[key]['p0_a20'] = total_a2_0/total_counts
            counts[key]['p0_a21'] = total_a2_1/total_counts
    PauliExpVal[theta] = counts


    return counts


In [7]:
# Calculates total energy of the Hamiltonian by summing up the expecation values of each Pauli term after
# multiplying with their respective coefficients
def totalEnergy_a20(theta:float, paulis_ev:dict):

    '''
    Takes as input theta value and associated pauli expectation values from the dictionary output of PauliEnergies function
    Calculates the total energy of the Hamiltonian by summing up the individual Pauli term expectation values
    for ancilla 2 = 0, after multiplying each value by their respective cofficients

    Returns total energy
    '''
    energy = paulis_ev[theta]['I']['coefficient']
    for key, pauli_dict in paulis_ev[theta].items():
        # compute energies

#         print('energyini ', energy)
        energy += paulis_ev[theta][key]['coefficient'] * paulis_ev[theta][key]['a2_0 exp-val']
#         print('energy', energy)
    return energy

def totalEnergy_a21(theta:float, paulis_ev:dict):

    '''
    Takes as input theta value and associated pauli expectation values from the dictionary output of PauliEnergies function
    Calculates the total energy of the Hamiltonian by summing up the individual Pauli term expectation values
    for ancilla 2 = 1, after multiplying each value by their respective cofficients

    Returns total energy
    '''

    energy_rot = paulis_ev[theta]['I']['coefficient']
    for key, pauli_dict in paulis_ev[theta].items():
        # compute energies

#         print('energyini ', energy_rot)

        energy_rot += paulis_ev[theta][key]['coefficient'] * paulis_ev[theta][key]['a2_1 exp-val']
#         print('energy', energy_rot)
    return energy_rot

In [8]:
def OptEnergyOptParams(energy_dict:dict):
    '''
    Takes as input the dictionary output of energies from the totalEnergy_a20 or totalEnergy_a21 function
    and outputs the minimum energy and the associated parameter
    '''
    opt_energy = min(energy_dict.values())
    opt_params = min(energy_dict, key=energy_dict.get)
    return{'opt-energy': opt_energy,
           'opt-param': opt_params}

In [16]:
## find variance and standard error of the mean for energy estimate

def SW_Var_a20(results:dict):
    '''
    Takes dictionary with data for optimal parameter, for
    ancilla 2 = 0 or 1 and finds variance of the energy using
    the Satterthwaite correction to account for
    different sample sizes for X and Z pauli operators.
    Returns Standard Error of the Mean using counts for ancilla 2 = 0
    '''
    varCS_sum = 0.0
    var = 0.0
    for pauli in results.keys():
#         print(results[depol]['a2_0']['data'][pauli]['coefficient'])
        if pauli != 'I':
#             print('coeff ', results[pauli]['coefficient']**2,
#                   'ev ', results[pauli]['a2_0 exp-val']**2,
#                 'diff ev ', 1-(results[pauli]['a2_0 exp-val']**2),
#                 'sum ', sum(results[pauli]['a2_0 counts'].values()))
            varCS_sum += (results[pauli]['coefficient']**2)\
                             *(1-(results[pauli]['a2_0 exp-val']**2))\
                            /sum(results[pauli]['a2_0 counts'].values())
            var += (results[pauli]['coefficient']**2)\
                             *(1-(results[pauli]['a2_0 exp-val']**2))
    varCS = np.sqrt(varCS_sum)

    return varCS, var

def SW_Var_a21(results:dict):
    '''
    Takes dictionary with data for optimal parameter and optimized energy, for
    ancilla 2 = 0 or 1 and finds variance of the energy using
    the Satterthwaite correction to account for
    different sample sizes for X and Z pauli operators.
    Returns Standard Error of the Mean using counts for ancilla 2 = 1
    '''
    varCS_sum = 0.0
    var = 0.0
    for pauli in results.keys():
#         print(results[depol]['a2_0']['data'][pauli]['coefficient'])
        if pauli != 'I':
#             print('coeff ', results[pauli]['coefficient']**2,
#                   'ev ', results[pauli]['a2_1 exp-val']**2,
#                 'diff ev ', 1-(results[pauli]['a2_1 exp-val']**2),
#                 'sum ', sum(results[pauli]['a2_1 counts'].values()))
            varCS_sum += (results[pauli]['coefficient']**2)\
                             *(1-(results[pauli]['a2_1 exp-val']**2))\
                            /sum(results[pauli]['a2_1 counts'].values())
            var += (results[pauli]['coefficient']**2)\
                             *(1-(results[pauli]['a2_1 exp-val']**2))

    varCS = np.sqrt(varCS_sum)

    return varCS, var






In [31]:
### unzip file
# !unzip /content/encoded_counts_noiseless_1.zip
!unzip /content/encoded_counts_noisy_1.zip

Archive:  /content/encoded_counts_noisy_1.zip
   creating: encoded_counts_noisy_1/
  inflating: __MACOSX/._encoded_counts_noisy_1  
  inflating: encoded_counts_noisy_1/H1-1E_VQEH2_Enc_3parallelCircs_noisy_10000shots_run1_part1.pkl  
  inflating: __MACOSX/encoded_counts_noisy_1/._H1-1E_VQEH2_Enc_3parallelCircs_noisy_10000shots_run1_part1.pkl  
  inflating: encoded_counts_noisy_1/._H1-1E_VQEH2_Enc_3parallelCircs_noisy_10000shots_run1_part1.pkl  
  inflating: encoded_counts_noisy_1/H1-1E_VQEH2_Enc_3parallelCircs_noisy_10000shots_run1_part2.pkl  
  inflating: __MACOSX/encoded_counts_noisy_1/._H1-1E_VQEH2_Enc_3parallelCircs_noisy_10000shots_run1_part2.pkl  
  inflating: encoded_counts_noisy_1/._H1-1E_VQEH2_Enc_3parallelCircs_noisy_10000shots_run1_part2.pkl  
  inflating: encoded_counts_noisy_1/H1-1E_VQEH2_Enc_3parallelCircs_noisy_10000shots_run1_part3.pkl  
  inflating: __MACOSX/encoded_counts_noisy_1/._H1-1E_VQEH2_Enc_3parallelCircs_noisy_10000shots_run1_part3.pkl  
  inflating: encoded_co

In [33]:
### delete extra files generated during unzip
!find /content/encoded_counts_noiseless_1 -type f -name '._*' -delete
!find /content/encoded_counts_noisy_1 -type f -name '._*' -delete

**Workflow begins here**

In [34]:
import os
import pickle

# Function to create list of results with the result from each file as elements
def read_pickle_files_to_list(directory):
    all_results = []

    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)

        # Check if it is a file and if it ends with .pkl (or other pickle extension)
        if os.path.isfile(file_path) and file_path.endswith('.pkl'):
            with open(file_path, 'rb') as file:
                results = pickle.load(file)
                all_results.append(results)

    return all_results

# Specify the directory containing the pickle files
# directory_path = '/content/encoded_counts_noiseless_1'
directory_path = '/content/encoded_counts_noisy_1'

# Read pickle files and get the list of contents
all_results = read_pickle_files_to_list(directory_path)

# Print the contents of each pickle file
# for index, content in enumerate(all_results):
#     print(f"Content of pickle file {index + 1}:\n{content}\n")


In [35]:
len(all_results)


7

In [36]:
# function to combine all counts within a file after splitting
# combined bitstring to get bitstrings for each circuit in a parallel run
def get_total_counts_per_file(splitcounts, points):
    for point in points:
        for key, terms in splitcounts[point].items():
            splitcounts[point][key]['MeasurementCounts'] = check_and_combine(terms['OriginalCounts'])

    return splitcounts



In [37]:
total_ech_file = []
# points_test = [-0.400606, -0.358437, -0.316268, -0.274099, -0.23193 , -0.189761, -0.147592, -0.105423, -0.063254, -0.021085]
points_test = [-0.22967]
# points_test = [-0.23193]
for result in all_results:
    combined_results = {}
    combined_results = get_total_counts_per_file(result, points_test)
    total_ech_file.append(combined_results)
# get_total_counts_per_file(all_results[0], points_test)


In [38]:
len(total_ech_file)

7

In [39]:
# Create a results_test dictionary from an element of total_ech_file to
# retain structure of dictionary for post-selection steps
results_test = total_ech_file[0].copy()

In [40]:
# test that sums of measurement counts are same or different for the one file with <10000 shots
for i in range(len(total_ech_file)):
    print(i)
    if i < len(total_ech_file)-1:
        print(sum(total_ech_file[i][-0.22967]['z1z2']['MeasurementCounts'].values())==sum(total_ech_file[i+1][-0.22967]['z1z2']['MeasurementCounts'].values()))

# len(total_ech_file)

0
False
1
False
2
True
3
True
4
True
5
True
6


In [41]:
# Counts list creates a lits of bitstring:counts dictionaries for each point and each pauli from all the files
for point in points_test:
    for pauli in ['I', 'z1z2', 'z1z3','z2z3', 'x2x3']:

         results_test[point][pauli]['counts_list'] = counts_list(total_ech_file, point, pauli)

# Combines list of counts dictionaries for each point and each pauli
for point in points_test:
    for pauli in ['I', 'z1z2', 'z1z3', 'z2z3', 'x2x3']:
        results_test[point][pauli]['MeasurementCounts'] = meta_combine(results_test[point][pauli]['counts_list'])

In [ ]:
## Save combined results in a file
# pickle.dump(results_test, open('VQEenc_Counts_Quantinuum_combined188001shots_h1-1E.pkl', 'wb'))


**Workflow: Post-selection for [[4,2,2]] code**

In [42]:
##### Workflow continued

## NO POST-SELECTION

shot = 188001
final_data = {'a2_0':{}, 'a2_1':{}}
# collect the expectation values of all the pauli terms in this dictionary
pauli_expval_test = {}
# collect values of total energy by value of theta in these dictionaries based on ancilla 2 value = 0 or 1
thetaEVa20_test = {}
thetaEVa21_test = {}

# collect values of minimum energy and correspinding optimal parameter for each dictionary containing total
# energy separated by ancilla 2 values
a20_noisy_exp_val = {}
a21_noisy_exp_val = {}
#loop over points and generate the dictionary with parameters and associated measurement counts

for point in points_test:

    # loop over keys and dictionaries of the pauli terms for each parameter and get correct counts
    # by discarding measurements with a1 = 1
    for key, terms in results_test[point].items():

        results_test[point][key]['correctCounts'] = results_test[point][key]['MeasurementCounts']

    # find the expectation values for each Pauli term for each parameter separated by the value of ancilla 2
    pauli_expval_test[point] = PauliEnergies(point, results_test[point], False)

    # find total energy energy by summing exp-val of all Pauli terms for each parameter but separated by ancilla value
    thetaEVa20_test[point] = totalEnergy_a20(point, pauli_expval_test)
    thetaEVa21_test[point] = totalEnergy_a21(point, pauli_expval_test)

#     with open(f'422EncodingVQE_a20_{depol}noise_{shot}shots_CSQtuum_Compare_thetaVsE.txt', 'w') as f:
#         f.write(json.dumps(thetaEVa20_test))

#     with open(f'422EncodingVQE_a21_{depol}noise_{shot}shots_CSQtuum_Compare_thetaVsE.txt', 'w') as f:
#         f.write(json.dumps(thetaEVa21_test))
# find minimum energy and optimal parameter for each ancilla 2 value by depolarizing parameter
results_a20_test = OptEnergyOptParams(thetaEVa20_test)
results_a21_test = OptEnergyOptParams(thetaEVa21_test)

# get minimum energy and optimal parameter for each depolarizing parameter
a20_noisy_exp_val = results_a20_test
a21_noisy_exp_val = results_a21_test
#     print(results_a20_test)
# calculate SEM
a20_sem, a20_var = SW_Var_a20(results_test[results_a20_test['opt-param']])
a21_sem, a21_var = SW_Var_a21(results_test[results_a21_test['opt-param']])

# collect results associated with optimal paramter
final_data['a2_0']['exp-val'] = results_a20_test['opt-energy']
final_data['a2_0']['data'] = results_test[results_a20_test['opt-param']]
final_data['a2_1']['exp-val'] = results_a21_test['opt-energy']
final_data['a2_1']['data'] = results_test[results_a21_test['opt-param']]
final_data['a2_0']['var'] = a20_var
final_data['a2_1']['var'] = a21_var
final_data['a2_0']['sem'] = a20_sem
final_data['a2_1']['sem'] = a21_sem
a20_noisy_exp_val['var'] = final_data['a2_0']['var']
a21_noisy_exp_val['var'] = final_data['a2_1']['var']
a20_noisy_exp_val['sem'] = final_data['a2_0']['sem']
a21_noisy_exp_val['sem'] = final_data['a2_1']['sem']
a20_noisy_exp_val['p0_z'] = final_data['a2_0']['data']['z1z2']['p0_a20']
a20_noisy_exp_val['p0_x'] = final_data['a2_0']['data']['x2x3']['p0_a20']
a21_noisy_exp_val['p0_z'] = final_data['a2_1']['data']['z1z2']['p0_a21']
a21_noisy_exp_val['p0_x'] = final_data['a2_1']['data']['x2x3']['p0_a21']
final_data['a2_0']['all-theta'] = thetaEVa20_test
final_data['a2_1']['all-theta'] = thetaEVa21_test

# ### SAVE data
# import json

# import pickle

# pickle.dump(final_data, open(f'VQEenc_results_noisy_{shot}shots_noPS_LSB_H1-1E.pkl', 'wb'))

# with open(f'422EncodingVQE_a20_noisy_{shot}shots_noPS_LSB_H1-1E.txt', 'w') as f:
#     f.write(json.dumps(a20_noisy_exp_val))

# with open(f'422EncodingVQE_a21_noisy_{shot}shots_noPS_LSB_H1-1E.txt', 'w') as f:
#     f.write(json.dumps(a21_noisy_exp_val))

In [43]:

a20_noisy_exp_val
# thetaEVa20_test
# final_data['a2_0']
# results_a20_test

{'opt-energy': -1.1182957344535067,
 'opt-param': -0.22967,
 'var': 0.06053630806897241,
 'sem': 0.0008009981756164023,
 'p0_z': 0.5022473284716571,
 'p0_x': 0.5015239280642124}

In [44]:
## POST-SELECTION BY ANCILLA A1=0

shot = 188001
final_data = {'a2_0':{}, 'a2_1':{}}
# collect the expectation values of all the pauli terms in this dictionary
pauli_expval_test = {}
# collect values of total energy by value of theta in these dictionaries based on ancilla 2 value = 0 or 1
thetaEVa20_test = {}
thetaEVa21_test = {}

# collect values of minimum energy and correspinding optimal parameter for each dictionary containing total
# energy separated by ancilla 2 values
a20_noisy_exp_val = {}
a21_noisy_exp_val = {}


#loop over every parameter to find expectation values for each

for point in points_test:

    # loop over keys and dictionaries of the pauli terms for each parameter and get correct counts
    # by discarding measurements with a1 = 1
    for key, terms in results_test[point].items():

        # remove counts with ancilla a1 = 0
        results_test[point][key]['correctCounts'] = post_select_on_ancila_1(terms['MeasurementCounts'], False)
#         results_test[point][key]['correctCounts'] = terms['MeasurementCounts']

    # find the expectation values for each Pauli term for each parameter separated by the value of ancilla 2
    pauli_expval_test[point] = PauliEnergies(point, results_test[point], False)

    # find total energy energy by summing exp-val of all Pauli terms for each parameter but separated by ancilla value
    thetaEVa20_test[point] = totalEnergy_a20(point, pauli_expval_test)
    thetaEVa21_test[point] = totalEnergy_a21(point, pauli_expval_test)

#     with open(f'422EncodingVQE_a20_{depol}noise_{shot}shots_CSQtuum_Compare_thetaVsE.txt', 'w') as f:
#         f.write(json.dumps(thetaEVa20_test))

#     with open(f'422EncodingVQE_a21_{depol}noise_{shot}shots_CSQtuum_Compare_thetaVsE.txt', 'w') as f:
#         f.write(json.dumps(thetaEVa21_test))
# find minimum energy and optimal parameter for each ancilla 2 value by depolarizing parameter
results_a20_test = OptEnergyOptParams(thetaEVa20_test)
results_a21_test = OptEnergyOptParams(thetaEVa21_test)

# get minimum energy and optimal parameter for each depolarizing parameter
a20_noisy_exp_val = results_a20_test
a21_noisy_exp_val = results_a21_test
#     print(results_a20_test)
# calculate SEM
a20_sem, a20_var = SW_Var_a20(results_test[results_a20_test['opt-param']])
a21_sem, a21_var = SW_Var_a21(results_test[results_a21_test['opt-param']])

# collect results associated with optimal paramter
final_data['a2_0']['exp-val'] = results_a20_test['opt-energy']
final_data['a2_0']['data'] = results_test[results_a20_test['opt-param']]
final_data['a2_1']['exp-val'] = results_a21_test['opt-energy']
final_data['a2_1']['data'] = results_test[results_a21_test['opt-param']]
final_data['a2_0']['var'] = a20_var
final_data['a2_1']['var'] = a21_var
final_data['a2_0']['sem'] = a20_sem
final_data['a2_1']['sem'] = a21_sem
a20_noisy_exp_val['var'] = final_data['a2_0']['var']
a21_noisy_exp_val['var'] = final_data['a2_1']['var']
a20_noisy_exp_val['sem'] = final_data['a2_0']['sem']
a21_noisy_exp_val['sem'] = final_data['a2_1']['sem']
a20_noisy_exp_val['p0_z'] = final_data['a2_0']['data']['z1z2']['p0_a20']
a20_noisy_exp_val['p0_x'] = final_data['a2_0']['data']['x2x3']['p0_a20']
a21_noisy_exp_val['p0_z'] = final_data['a2_1']['data']['z1z2']['p0_a21']
a21_noisy_exp_val['p0_x'] = final_data['a2_1']['data']['x2x3']['p0_a21']
final_data['a2_0']['all-theta'] = thetaEVa20_test
final_data['a2_1']['all-theta'] = thetaEVa21_test


### SAVE data

# import json

# import pickle

# pickle.dump(final_data, open(f'VQEenc_results_noisy_{shot}shots_PSA_LSB_H1-1E.pkl', 'wb'))

# with open(f'422EncodingVQE_a20_noisy_{shot}shots_PSA_LSB_H1-1E.txt', 'w') as f:
#     f.write(json.dumps(a20_noisy_exp_val))

# with open(f'422EncodingVQE_a21_noisy_{shot}shots_PSA_LSB_H1-1E.txt', 'w') as f:
#     f.write(json.dumps(a21_noisy_exp_val))

In [45]:
a20_noisy_exp_val
# final_data['a2_0']['data']['z1z2']['correctCounts']

{'opt-energy': -1.1203647393920662,
 'opt-param': -0.22967,
 'var': 0.05898636066274143,
 'sem': 0.0007927892502720783,
 'p0_z': 0.49968351232174296,
 'p0_x': 0.49877926181243715}

In [46]:
## POST-SELECTION BY PARITY

shot = 188001
final_data = {'a2_0':{}, 'a2_1':{}}
# collect the expectation values of all the pauli terms in this dictionary
pauli_expval_test = {}
# collect values of total energy by value of theta in these dictionaries based on ancilla 2 value = 0 or 1
thetaEVa20_test = {}
thetaEVa21_test = {}

# collect values of minimum energy and correspinding optimal parameter for each dictionary containing total
# energy separated by ancilla 2 values
a20_noisy_exp_val = {}
a21_noisy_exp_val = {}

#loop over points and generate the dictionary with parameters and associated measurement counts

for point in points_test:

    # loop over keys and dictionaries of the pauli terms for each parameter and get correct counts
    # by discarding measurements with a1 = 1
    for key, terms in results_test[point].items():

        # For parity only post-selection, uncomment this and comment out above
        results_test[point][key]['correctCounts'] = post_select_codespace(results_test[point][key]['MeasurementCounts'], False)

    # find the expectation values for each Pauli term for each parameter separated by the value of ancilla 2
    pauli_expval_test[point] = PauliEnergies(point, results_test[point], False)

    # find total energy energy by summing exp-val of all Pauli terms for each parameter but separated by ancilla value
    thetaEVa20_test[point] = totalEnergy_a20(point, pauli_expval_test)
    thetaEVa21_test[point] = totalEnergy_a21(point, pauli_expval_test)

#     with open(f'422EncodingVQE_a20_{depol}noise_{shot}shots_CSQtuum_Compare_thetaVsE.txt', 'w') as f:
#         f.write(json.dumps(thetaEVa20_test))

#     with open(f'422EncodingVQE_a21_{depol}noise_{shot}shots_CSQtuum_Compare_thetaVsE.txt', 'w') as f:
#         f.write(json.dumps(thetaEVa21_test))
# find minimum energy and optimal parameter for each ancilla 2 value by depolarizing parameter
results_a20_test = OptEnergyOptParams(thetaEVa20_test)
results_a21_test = OptEnergyOptParams(thetaEVa21_test)

# get minimum energy and optimal parameter for each depolarizing parameter
a20_noisy_exp_val = results_a20_test
a21_noisy_exp_val = results_a21_test
#     print(results_a20_test)
# calculate SEM
a20_sem, a20_var = SW_Var_a20(results_test[results_a20_test['opt-param']])
a21_sem, a21_var = SW_Var_a21(results_test[results_a21_test['opt-param']])

# collect results associated with optimal paramter
final_data['a2_0']['exp-val'] = results_a20_test['opt-energy']
final_data['a2_0']['data'] = results_test[results_a20_test['opt-param']]
final_data['a2_1']['exp-val'] = results_a21_test['opt-energy']
final_data['a2_1']['data'] = results_test[results_a21_test['opt-param']]
final_data['a2_0']['var'] = a20_var
final_data['a2_1']['var'] = a21_var
final_data['a2_0']['sem'] = a20_sem
final_data['a2_1']['sem'] = a21_sem
a20_noisy_exp_val['var'] = final_data['a2_0']['var']
a21_noisy_exp_val['var'] = final_data['a2_1']['var']
a20_noisy_exp_val['sem'] = final_data['a2_0']['sem']
a21_noisy_exp_val['sem'] = final_data['a2_1']['sem']
a20_noisy_exp_val['p0_z'] = final_data['a2_0']['data']['z1z2']['p0_a20']
a20_noisy_exp_val['p0_x'] = final_data['a2_0']['data']['x2x3']['p0_a20']
a21_noisy_exp_val['p0_z'] = final_data['a2_1']['data']['z1z2']['p0_a21']
a21_noisy_exp_val['p0_x'] = final_data['a2_1']['data']['x2x3']['p0_a21']
final_data['a2_0']['all-theta'] = thetaEVa20_test
final_data['a2_1']['all-theta'] = thetaEVa21_test


### SAVE data

# import json

# import pickle

# pickle.dump(final_data, open(f'VQEenc_results_noisy_{shot}shots_PSP_LSB_H1-1E.pkl', 'wb'))

# with open(f'422EncodingVQE_a20_noisy_{shot}shots_PSP_LSB_H1-1E.txt', 'w') as f:
#     f.write(json.dumps(a20_noisy_exp_val))

# with open(f'422EncodingVQE_a21_noisy_{shot}shots_PSP_LSB_H1-1E.txt', 'w') as f:
#     f.write(json.dumps(a21_noisy_exp_val))

In [47]:
a20_noisy_exp_val

{'opt-energy': -1.1288046439901662,
 'opt-param': -0.22967,
 'var': 0.05290760022857842,
 'sem': 0.0007544292378186278,
 'p0_z': 0.4953324716357892,
 'p0_x': 0.4938431178557561}

In [48]:
## POST SELECTION BY COMBINED METHOD

shot = 188001
final_data = {'a2_0':{}, 'a2_1':{}}
# collect the expectation values of all the pauli terms in this dictionary
pauli_expval_test = {}
# collect values of total energy by value of theta in these dictionaries based on ancilla 2 value = 0 or 1
thetaEVa20_test = {}
thetaEVa21_test = {}

# collect values of minimum energy and correspinding optimal parameter for each dictionary containing total
# energy separated by ancilla 2 values
a20_noisy_exp_val = {}
a21_noisy_exp_val = {}

#loop over points and generate the dictionary with parameters and associated measurement counts

for point in points_test:

    # loop over keys and dictionaries of the pauli terms for each parameter and get correct counts
    # by discarding measurements with a1 = 1
    for key, terms in results_test[point].items():

        # remove counts with ancilla a1 = 0
        results_test[point][key]['correctCounts'] = post_select_on_ancila_1(terms['MeasurementCounts'], False)
#             # remove counts with bitstrings not included in the codespace + ancilla a1=0
        results_test[point][key]['correctCounts'] = post_select_codespace(results_test[point][key]['correctCounts'], False)

    # find the expectation values for each Pauli term for each parameter separated by the value of ancilla 2
    pauli_expval_test[point] = PauliEnergies(point, results_test[point], False)

    # find total energy energy by summing exp-val of all Pauli terms for each parameter but separated by ancilla value
    thetaEVa20_test[point] = totalEnergy_a20(point, pauli_expval_test)
    thetaEVa21_test[point] = totalEnergy_a21(point, pauli_expval_test)

#     with open(f'422EncodingVQE_a20_{depol}noise_{shot}shots_CSQtuum_Compare_thetaVsE.txt', 'w') as f:
#         f.write(json.dumps(thetaEVa20_test))

#     with open(f'422EncodingVQE_a21_{depol}noise_{shot}shots_CSQtuum_Compare_thetaVsE.txt', 'w') as f:
#         f.write(json.dumps(thetaEVa21_test))
# find minimum energy and optimal parameter for each ancilla 2 value by depolarizing parameter
results_a20_test = OptEnergyOptParams(thetaEVa20_test)
results_a21_test = OptEnergyOptParams(thetaEVa21_test)

# get minimum energy and optimal parameter for each depolarizing parameter
a20_noisy_exp_val = results_a20_test
a21_noisy_exp_val = results_a21_test
#     print(results_a20_test)
# calculate SEM
a20_sem, a20_var = SW_Var_a20(results_test[results_a20_test['opt-param']])
a21_sem, a21_var = SW_Var_a21(results_test[results_a21_test['opt-param']])

# collect results associated with optimal paramter
final_data['a2_0']['exp-val'] = results_a20_test['opt-energy']
final_data['a2_0']['data'] = results_test[results_a20_test['opt-param']]
final_data['a2_1']['exp-val'] = results_a21_test['opt-energy']
final_data['a2_1']['data'] = results_test[results_a21_test['opt-param']]
final_data['a2_0']['var'] = a20_var
final_data['a2_1']['var'] = a21_var
final_data['a2_0']['sem'] = a20_sem
final_data['a2_1']['sem'] = a21_sem
a20_noisy_exp_val['var'] = final_data['a2_0']['var']
a21_noisy_exp_val['var'] = final_data['a2_1']['var']
a20_noisy_exp_val['sem'] = final_data['a2_0']['sem']
a21_noisy_exp_val['sem'] = final_data['a2_1']['sem']
a20_noisy_exp_val['p0_z'] = final_data['a2_0']['data']['z1z2']['p0_a20']
a20_noisy_exp_val['p0_x'] = final_data['a2_0']['data']['x2x3']['p0_a20']
a21_noisy_exp_val['p0_z'] = final_data['a2_1']['data']['z1z2']['p0_a21']
a21_noisy_exp_val['p0_x'] = final_data['a2_1']['data']['x2x3']['p0_a21']
final_data['a2_0']['all-theta'] = thetaEVa20_test
final_data['a2_1']['all-theta'] = thetaEVa21_test

### SAVE data

# import json

# import pickle

# pickle.dump(final_data, open(f'VQEenc_results_noisy_{shot}shots_PSAP_LSB_H1-1E.pkl', 'wb'))

# with open(f'422EncodingVQE_a20_noisy_{shot}shots_PSAP_LSB_H1-1E.txt', 'w') as f:
#     f.write(json.dumps(a20_noisy_exp_val))

# with open(f'422EncodingVQE_a21_noisy_{shot}shots_PSAP_LSB_H1-1E.txt', 'w') as f:
#     f.write(json.dumps(a21_noisy_exp_val))

In [49]:
a20_noisy_exp_val

{'opt-energy': -1.129784831552889,
 'opt-param': -0.22967,
 'var': 0.052181350137215456,
 'sem': 0.000750760204030219,
 'p0_z': 0.4933324822740305,
 'p0_x': 0.49184844761464036}